# Building the Agent 

The agent is comprised of a router using OpenAI function calling, and a set of three tools: a database lookup tool, a data analysis tool, and a code generator to create graphs.

<img src="images/Agent.png" width="1000"/>


The agent can lookup information from a local file, perform analysis on that information, and graph results. The example local file is a log of transactions at a local store. The agent can help the store owners understand trends and anomalies in their sales data.

## Importing necessary libraries 

In [1]:
from openai import OpenAI
import pandas as pd
import json
import duckdb
from pydantic import BaseModel, Field
from IPython.display import Markdown
from helper import get_openai_api_key

## Initializing the OpenAI client

In [2]:
# initialize the OpenAI client
openai_api_key = get_openai_api_key()
client = OpenAI(api_key=openai_api_key)

MODEL = "gpt-4o-mini"

## Defining the tools

Let's start by creating the three tools the agent will be able to use.

### Tool 1: Database Lookup

This first tool reads from a local parquet file that contains the transaction data. 

In [3]:
# define the path to the transactional data
TRANSACTION_DATA_FILE_PATH = 'data/Store_Sales_Price_Elasticity_Promotions_Data.parquet'

In [4]:
df = pd.read_parquet(TRANSACTION_DATA_FILE_PATH)
df.head(10) 

,Store_Number,SKU_Coded,Product_Class_Code,Sold_Date,Qty_Sold,Total_Sale_Value,On_Promo
0,1320,6172800,22875,2021-11-02,3,56.849998,0
1,2310,6172800,22875,2021-11-03,1,18.950001,0
2,3080,6172800,22875,2021-11-03,1,18.950001,0
3,2310,6172800,22875,2021-11-06,1,18.950001,0
4,4840,6172800,22875,2021-11-07,1,18.950001,0
5,1320,6172800,22875,2021-11-08,1,18.950001,0
6,2310,6172800,22875,2021-11-08,2,37.900002,0
7,330,6172800,22875,2021-11-10,1,18.950001,0
8,2200,6172800,22875,2021-11-10,1,18.950001,0
9,2090,6172800,22875,2021-11-14,1,18.950001,0


This database lookup tool works using three steps. 

<img src="images/tool1.png" width="500"/>

1. First, it creates the SQL table from a local file, if not already done.
2. Second, it translates the original prompt into an sql query (using an LLM call).
3. Finally, it runs that query against the database.

In [5]:
# prompt template for step 2 of tool 1
SQL_GENERATION_PROMPT = """
Generate an SQL query based on a prompt. Do not reply with anything besides the SQL query.
The prompt is: {prompt}

The available columns are: {columns}
The table name is: {table_name}
"""

In [6]:
# code for step 2 of tool 1
def generate_sql_query(prompt: str, columns: list, table_name: str) -> str:
    """Generate an SQL query based on a prompt"""
    formatted_prompt = SQL_GENERATION_PROMPT.format(prompt=prompt, 
                                                    columns=columns, 
                                                    table_name=table_name)

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    
    return response.choices[0].message.content

In [7]:
# code for tool 1
def lookup_sales_data(prompt: str) -> str:
    """Implementation of sales data lookup from parquet file using SQL"""
    try:

        # define the table name
        table_name = "sales"
        
        # step 1: read the parquet file into a DuckDB table
        df = pd.read_parquet(TRANSACTION_DATA_FILE_PATH)
        duckdb.sql(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM df")

        # step 2: generate the SQL code
        sql_query = generate_sql_query(prompt, df.columns, table_name)
        print ('Generated SQL query =', sql_query)
        # clean the response to make sure it only includes the SQL code
        sql_query = sql_query.strip()
        sql_query = sql_query.replace("```sql", "").replace("```", "")
        
        # step 3: execute the SQL query
        result = duckdb.sql(sql_query).df()
        
        return result.to_string()
    except Exception as e:
        return f"Error accessing data: {str(e)}"

Great! Now let's test the first tool and make sure it worked correctly.

In [ ]:
example_data = lookup_sales_data("Extract all transactions for store 1320 for the 21 st of November 2022")
print(example_data)

### Tool 2: Data Analysis

The second tool can analyze the returned data and display conclusions to users.

<img src="images/tool2.png" width="300"/>

In [9]:
# Construct prompt based on analysis type and data subset
DATA_ANALYSIS_PROMPT = """
Analyze the following data: {data}
Your job is to answer the following question: {prompt}
"""

In [10]:
# code for tool 2
def analyze_sales_data(prompt: str, data: str) -> str:
    """Implementation of AI-powered sales data analysis"""
    formatted_prompt = DATA_ANALYSIS_PROMPT.format(data=data, prompt=prompt)
    

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    
    analysis = response.choices[0].message.content
    return analysis if analysis else "No analysis could be generated"

This tool is relatively simple, but let's still test it out to be sure things are working correctly.

In [11]:
print(analyze_sales_data(prompt="what trends do you see in this data", 
                         data=example_data))

Analyzing the data provided, we can identify several trends and insights based on sales activities. The following are key observations:

### Overall Sales Trends
1. **Sales Over Time**: There are multiple sales recorded over the month of November 2022, with certain days showing higher transaction volumes and quantities sold indicating potential peak sales periods. This can be further analyzed by aggregating sales by date to identify peak sales days.

2. **Product Class Code**: The dataset has various product class codes (22800, 22875, 24425, etc.), allowing us to explore which product classes are selling the most. A frequency count of `Product_Class_Code` in conjunction with total sales value can reveal which products are performing best.

3. **Quantity Sold**: The dataset shows varying quantities sold for different SKUs. Some products have high transaction volumes (e.g., 12 items sold in one transaction) while many others reflect smaller sales, suggesting a mix of high-demand and low-

## Defining the Router

Now that all of the tools are defined, you can create the router. The router will take the original user input, and is responsible for calling any tools. After each tool call is completed, the agent will return to router to determine whether another tool should be called.

### Tool Schema

Let's define the tools in a way that can be understood by our OpenAI model. OpenAI understands a specific JSON format:

In [13]:
# Define tools/functions that can be called by the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "lookup_sales_data",
            "description": "Look up data from Store Sales Price Elasticity Promotions dataset",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {"type": "string", "description": "The unchanged prompt that the user provided."}
                },
                "required": ["prompt"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_sales_data", 
            "description": "Analyze sales data to extract insights",
            "parameters": {
                "type": "object",
                "properties": {
                    "data": {"type": "string", "description": "The lookup_sales_data tool's output."},
                    "prompt": {"type": "string", "description": "The unchanged prompt that the user provided."}
                },
                "required": ["data", "prompt"]
            }
        }
    }
]

# Dictionary mapping function names to their implementations
tool_implementations = {
    "lookup_sales_data": lookup_sales_data,
    "analyze_sales_data": analyze_sales_data, 
}

### Router Logic

The router is composed of a main loop method, and a method to handle the tool calls that you get back from the model.

<img src="images/Agent.png" width="800"/>

The following two cells define the function `handle_tool_calls` and the variable `SYSTEM_PROMPT`, which will be used by the function `run_agent` defining the router logic.

In [14]:
# code for executing the tools returned in the model's response
def handle_tool_calls(tool_calls, messages):
    
    for tool_call in tool_calls:   
        function = tool_implementations[tool_call.function.name]
        function_args = json.loads(tool_call.function.arguments)
        result = function(**function_args)
        messages.append({"role": "tool", "content": result, "tool_call_id": tool_call.id})
        
    return messages

In [15]:
SYSTEM_PROMPT = """
You are a helpful assistant that can answer questions about the Store Sales Price Elasticity Promotions dataset.
"""

In [16]:
def run_agent(messages):
    print("Running agent with messages:", messages)

    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
        
    # Check and add system prompt if needed
    if not any(
            isinstance(message, dict) and message.get("role") == "system" for message in messages
        ):
            system_prompt = {"role": "system", "content": SYSTEM_PROMPT}
            messages.append(system_prompt)

    while True:
        print("Making router call to OpenAI")
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools,
        )
        messages.append(response.choices[0].message)
        tool_calls = response.choices[0].message.tool_calls
        print("Received response with tool calls:",tool_calls)

        # if the model decides to call function(s), call handle_tool_calls
        if tool_calls:
            print("Processing tool calls")
            messages = handle_tool_calls(tool_calls, messages)
        else:
            print("No tool calls, returning final response")
            return response.choices[0].message.content

In [17]:
result = run_agent('Analyze sales for the store 1320 for the year 2022')

Running agent with messages: Analyze sales for the store 1320 for the year 2022
Making router call to OpenAI
Received response with tool calls: [ChatCompletionMessageToolCall(id='call_TnqDHhcCfANPybdAKdx1onHF', function=Function(arguments='{"prompt":"Analyze sales for the store 1320 for the year 2022"}', name='lookup_sales_data'), type='function')]
Processing tool calls
Generated SQL query = ```sql
SELECT 
    SUM(Qty_Sold) AS Total_Quantity_Sold,
    SUM(Total_Sale_Value) AS Total_Sales_Value,
    COUNT(DISTINCT SKU_Coded) AS Unique_SKUs_Sold
FROM 
    sales
WHERE 
    Store_Number = 1320 
    AND YEAR(Sold_Date) = 2022;
```
Making router call to OpenAI
Received response with tool calls: [ChatCompletionMessageToolCall(id='call_Atl39z8MmJwNh7e5VQUKfNJR', function=Function(arguments='{"data":" Total_Quantity_Sold  Total_Sales_Value  Unique_SKUs_Sold\\n0              17013.0      223011.838803               473","prompt":"Analyze sales for the store 1320 for the year 2022"}', name='analy

In [18]:
print(result)
# you can also print a formatted version of the result
# Markdown(result)

For store 1320, the sales performance in 2022 shows the following key metrics:

1. **Total Quantity Sold**: 17,013 units
2. **Total Sales Value**: Approximately $223,011.84
3. **Unique SKUs Sold**: 473 SKUs

### Insights and Analysis:

- **Average Selling Price (ASP)**:
  - The Average Selling Price per unit can be calculated as:
    \[
    \text{ASP} = \frac{\text{Total Sales Value}}{\text{Total Quantity Sold}} = \frac{223011.84}{17013} \approx 13.13
    \]
  - This results in an ASP of approximately **$13.13**.

- **Sales Performance**:
  - The total quantity sold and the sales value indicate a healthy demand for the products offered. However, understanding sales trends compared to previous years or similar stores would give a clearer picture of performance.

- **Product Diversity**:
  - Selling 473 unique SKUs highlights a broad assortment of products, which is beneficial for targeting various customer segments.

### Recommendations:

1. **Inventory Management**:
   - Analyze sales 